**Parsing Notes**
1. The parsing states and transitions are not understood
2. Some entries seem to be missing context

In [355]:
from bs4 import BeautifulSoup as Soup
from IPython.display import display, HTML
import pandas as pd
import re
import pprint
import json

In [356]:
#test text
test_text = '<p> <strong>NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ' \
         'ACQUISITIONS AND DISPOSITIONS PUBLIC HEARING</strong>, in ' \
         'accordance with Section 824 of the New York City Charter, will be ' \
         'held on December 10, 2014 at 10:30 a.m., 22 Reade Street, ' \
         '2<sup>nd</sup> floor conference room, Borough of Manhattan, in the ' \
         'matter of a lease for the City of New York, as tenant, of ' \
         'approximately 16,334 rentable square feet of space on the entire ' \
         'fifth (5<sup>th</sup>) floor of the building located at 118-35 ' \
         'Queens Boulevard (Block 2270, Lot 41) in the Borough of Queens for ' \
         'a Computerized Testing and Applications Center, or for such other ' \
         'use as the Commissioner of the Department of Citywide ' \
         'Administrative Services may determine.</p> <p> &nbsp;</p> <p> The ' \
         'proposed use was approved by the City Planning Commission pursuant ' \
         'to NYC Charter Section 195 on May 7, 2014 (CPC Appl. No. 140293 ' \
         'PXQ, Public Hearing Cal. No. 10).</p> <p> &nbsp;</p> <p> The ' \
         'proposed lease shall be for a period of twenty (20) years from ' \
         'Substantial Completion of alterations and improvements, at an ' \
         'annual rent of $612,103.00 ($37.47 per square foot) for the first ' \
         'five (5) years, $685,555.36 ($41.97 per square foot) for the next ' \
         'five (5) years, $767,872.66 ($47.01 per square foot) for the next ' \
         'five (5) years, and $859,899.18 ($52.64 per square foot) for the ' \
         'final five (5) years, payable in equal monthly installments at the ' \
         'end of each month.&nbsp; The first six (6) months&rsquo; rent ' \
         'shall be abated.</p> <p> &nbsp;</p> <p> The lease may be ' \
         'terminated by the Tenant on the fifth (5<sup>th</sup>), tenth ' \
         '(10<sup>th</sup>), and fifteenth (15<sup>th</sup>) anniversaries ' \
         'of the commencement date, provided the Tenant gives the Landlord ' \
         'no less than twelve (12) months prior written notice.&nbsp; In the ' \
         'event that the lease is terminated by the Tenant, the Tenant shall ' \
         'pay the Landlord the unamortized portion of the Tenant&rsquo;s ' \
         'broker&rsquo;s commission, Tenant Improvement Allowance, and free ' \
         'rent.</p> <p> &nbsp;</p> <p> The Tenant shall have the right to ' \
         'renew the lease for a period of five (5) years at 95% of Fair ' \
         'Market Value (FMV).</p> <p> &nbsp;</p> <p> The Landlord shall ' \
         'prepare final architectural plans and engineering plans and make ' \
         'alterations and improvements in accordance with preliminary ' \
         'architectural plans and specifications which are attached to the ' \
         'lease. The alterations and improvements consist of Base Building ' \
         'Work, which the landlord shall provide at its sole cost and ' \
         'expense, and Tenant Work.&nbsp; The total cost of the Tenant Work ' \
         'shall not exceed $3,269,800 ($200.18 per rentable square foot) of ' \
         'which the Landlord shall contribute $714,280.00 ($43.73 per ' \
         'rentable square foot) and the balance up to 2,555,520.00 ($156.45 ' \
         'per rentable square foot) will be paid by the Tenant, to be ' \
         'disbursed in accordance with the terms of the lease.</p> <p> ' \
         '&nbsp;</p> <p> Further information, including public inspection of ' \
         'the proposed lease may be obtained at One Centre Street, Room 2000 ' \
         'North, New York, N.Y.&nbsp; 10007.&nbsp; To schedule an ' \
         'inspection, please contact Chris Fleming at (212) 386-0315.</p> ' \
         '<p> &nbsp;</p> <p> Individuals requesting Sign Language ' \
         'Interpreters should contact the Mayor&rsquo;s Office of Contract ' \
         'Services, Public Hearings Unit, 253 Broadway, 2<sup>nd</sup> ' \
         'Floor, New York, N.Y.&nbsp; 10007,&nbsp; (212) 788-7490, no later ' \
         'than SEVEN (7) BUSINESS DAYS PRIOR TO THE PUBLIC HEARING.&nbsp; ' \
         'TDD users should call VERIZON relay services.</p>'

In [370]:
class Token:
    def __init__(self, name, value=''):
        self.name = name
        self.value = value
    
    def __repr__(self):
        return '{} : {}'.format(self.name, self.value)
        
# we will use these tokens:
# NOTICE -> when NOTICE IS HEREBY GIVEN is detected
# MATTER -> when IN THE MATTER OF is detected

def tokenize(fragment):
    soup = Soup(fragment)
    state = None
    for para in soup.find_all('p'):
        text = para.get_text(strip=True)
        if state is None:
            if 'notice is hereby given' in text.lower():
                yield Token('NOTICE')
                if 'in the matter' in text.lower():
                    loc = text.lower().find('in the matter')
                    yield Token('TEXT', text[:loc])
                    yield Token('MATTER')
                    yield Token('TEXT', text[loc:])
                    state = 'MATTER'
                else:
                    yield Token('TEXT', text)
        elif state == 'MATTER':
            if 'in the matter' in text.lower():
                loc = text.lower().find('in the matter')
                yield Token('TEXT', text[:loc])
                yield Token('MATTER')
                yield Token('TEXT', text[loc:])
                state = 'MATTER'
    if state == 'MATTER':
        yield Token('MATTER')
                
            
for tok in tokenize(test_text):
    print(tok)
        
        
        

NOTICE : 
TEXT : NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC HEARING, in accordance with Section 824 of the New York City Charter, will be held on December 10, 2014 at 10:30 a.m., 22 Reade Street, 2ndfloor conference room, Borough of Manhattan, 
MATTER : 
TEXT : in the matter of a lease for the City of New York, as tenant, of approximately 16,334 rentable square feet of space on the entire fifth (5th) floor of the building located at 118-35 Queens Boulevard (Block 2270, Lot 41) in the Borough of Queens for a Computerized Testing and Applications Center, or for such other use as the Commissioner of the Department of Citywide Administrative Services may determine.
MATTER : 


In [371]:
def test(text):
    state = Init()
    soup = Soup(text)
    display(HTML(text))
    record = { 'matters' : [], 'context' : '' }
    for para in soup.find_all('p'):
        state.run()
        state = state.next(para, record)
    state.flush(record)
    pprint.pprint(record)
# test(test_text)    
    

In [372]:
fn = 'procPublicationRequest Oct-Dec 2014 (Updated) - Sheet1-2.csv'
rows = pd.read_csv(fn, header=0)

In [373]:
class State:
    def run(self):
        # implement this for printing trace statements
        assert 0, "implement me"
        
    def next(self, input):
        assert 0, "implement me"
        
    def flush(self, record):
        raise Exception('wtf!')


In [379]:
class Matters(State):
    
    def __init__(self):
        State.__init__(self)
        self.matter = ''
    
    def run(self):
        print('Process MATTERS')
        
    def flush(self, record):
        record['matters'].append({ 'matter' : self.matter })
        self.matter = ''
        
    def next(self, tok, record):
        if tok.name == 'TEXT':
            self.matter += tok.value
        if tok.name == 'MATTER':
            self.flush(record)
        return self
            

class Notice(State):
    
    def __init__(self, notice=''):
        State.__init__(self)
        self.notice = notice
        
    def run(self):
        print('NOTICE')
        
    def next(self, tok, record):
        if tok.name == 'TEXT':
            self.notice += tok.value
            return self
        if tok.name == 'MATTER':
            record['context'] = self.notice
            return Matters()
        return self
    
    def flush(self, record):
        record['error'] = 'no transition out of NOTICE'
        
    
class Init(State):
    
    def run(self):
        print('INIT')
        
    def next(self, tok, record):
        if tok.name == 'NOTICE':
            return Notice()
        return self
                                  
    def flush(self, record):
        record['error'] = 'no transition out of INIT'
                                  

In [380]:
agency = rows['AgencyName'] == "Citywide Administrative Services"
description = rows['TypeOfNoticeDescription'] == "Public Hearings"
target_rows = rows[agency & description]


In [381]:
# def scrape(row):
#     output = {}
#     if not isinstance(row.AdditionalDescription, str):
#         output = { 'error' : 'source is not a string: {}'.format(row.AdditionalDescription) }
#     else:    
#         soup = Soup(row.AdditionalDescription)
#         state = Init()
#         record = { 'matters' :[], 'context' : '' }
#         for para in soup.find_all('p'):
#             state = state.next(para, record)
#         state.flush(record)
#         output = record

#     row['output'] = json.dumps(output)
#     return row

In [382]:
def scrape(row):
    output = {}
    if not isinstance(row.AdditionalDescription, str):
        output = { 'error' : 'source is not a string: {}'.format(row.AdditionalDescription) }
    else:   
        state = Init()
        record = { 'matters' :[], 'context' : '' }
        for tok in tokenize(row.AdditionalDescription):
            state = state.next(tok, record)
        output = record

    row['output'] = json.dumps(output)
    return row

In [383]:
processed_rows = target_rows.apply(scrape,1)

In [384]:
errors = []
for rec in processed_rows[['RequestID', 'output', 'AdditionalDescription']].values:
    id, output, desc = rec
    output = json.loads(output)
    if output.get('error', None):
        errors.append({'RequestID' : id, 'error': output, 'desc' : desc})
        continue
    print('RequestID: {}'.format(id))
#     display(HTML(desc))
    pprint.pprint(output)
    
#     print('\n\n')
#     pprint.pprint(Soup(desc).prettify())
#     print('\n\n')
    display(HTML('<hr/>'))

RequestID: 20140930111
{'context': '', 'matters': []}


RequestID: 20141006102
{'context': '', 'matters': []}


RequestID: 20141007108
{'context': '', 'matters': []}


RequestID: 20141105101
{'context': '', 'matters': []}


RequestID: 20141106101
{'context': '', 'matters': []}


...

RequestID: 20141201104
{'context': '', 'matters': []}


RequestID: 20141201109
{'context': '', 'matters': []}


RequestID: 20141219102
{'context': '', 'matters': []}


RequestID: 20141219106
{'context': '', 'matters': []}


RequestID: 20141223103
{'context': '', 'matters': []}


In [385]:
err_summary = '<h1>{} Errors parsing Citywide Administrative Services::Public Hearings'.format(len(errors))
display(HTML(err_summary))
for error in errors:
    pprint.pprint(error)
    print('\n\n')
    desc = error['desc']
#     if isinstance(desc, str):
#         pprint.pprint(Soup(error['desc']).prettify())
#     print('\n\n')


{'RequestID': 20141030107,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}



{'RequestID': 20141031112,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}



{'RequestID': 20141103110,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}



{'RequestID': 20141120108,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}



{'RequestID': 20141128101,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}



{'RequestID': 20141203109,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}



